In [1]:
import pandas as pd
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
from tqdm import tqdm 
from urllib.parse import urljoin
db_pomoci = pd.read_csv("db_pomoci.csv")

In [2]:
db_pomoci = pd.read_csv("db_pomoci.csv")
black_list = ['http://www.dc-brno.cz']
df = db_pomoci.rename(columns={"Webová stránka" : "web", "Název":"nazev"})
df = df[(~df["web"].isna()) & (~df['web'].isin(black_list))]
df.loc[df['web'].str.startswith('www'), 'web'] = df['web'].str.replace('^www', 'https://www', regex=True)
# df['web'] = df['web'].str.replace('https://www', 'www')
# df['web'] = df['web'].str.replace('www', 'https://www')
df_agg = df.groupby("web")["Telefon"].agg(list)
df_agg["email"] = df.groupby("web")["E-mail"].agg(list)
df_agg = df_agg.reset_index().rename(columns={'web': 'web', 'Telefon': 'Telefon'})
df_agg = pd.DataFrame(df_agg)
#df.sort_values("web")


In [22]:
db_scraped = pd.read_csv("data/df_scraped.csv")
df_res = db_scraped[~db_scraped["Phone Numbers"].isna()]
df_res = df_res.rename(columns={'web': 'web', 'Telefon': 'Telefon'}).drop("Unnamed: 0", axis= 1)
df_res = df_res.rename(columns= {"Website":"web"})
df_res = df_res.rename(columns= {"Phone Numbers":"phones_scraped"})

In [23]:
preview = df_res.merge(df_agg , on="web")
preview['phones_scraped'] = preview['phones_scraped'].str.split(', ')
preview

,web,phones_scraped,Telefon
0,http://www.pdz.cz,"[121 407 9329, 64 24 22 18, 420 466 400 116]","[778 526 216, 469 626 383, 774 915 755]"
1,http://www.medsix.cz/?option=com_content&view=...,"[483 341 232, 483 300 087, 483 310 919, 483 34...",[483 341 267]
2,https://jindrichuvhradec.charita.cz/sluzba/852/,"[731 402 982, 731 604 554, 605 849 877]","[731 402 982, 605 849 877]"
3,https://www.homeopatiecz.com/,"[777 169 278, 736 660 315, 777 146 760, 725 76...",[736 660 315]
4,http://www.ponton.cz/,[773 334 363],[774 433 395]
...,...,...,...
557,https://www.firmy.cz/detail/13179143-phdr-tere...,[420 376 322 688],[376 322 688]
558,https://search.seznam.cz/?q=psychiatrick%C3%A1...,[420608971030],[736 132 143]
559,http://ssp-ol.cz/,[585 427 141],[774 406 453]
560,http://www.psychoterapeuti.cz/adresar-psychote...,"[284 016 451, 736 660 315, 530 334 600, 775 18...",[466 611 601]


In [3]:
#df[df["web"].str.contains('brno')]

,nazev,Kategorie,Adresa,E-mail,Telefon,web
63,CDZ Brno I (Práh jižní Morava),CDZ - Centrum duševního zdraví,"Štěpánská 2, Brno-Trnitá (část) 602 00, Jihomo...",terennitym@prah-brno.cz,733 643 379,https://www.cdz-brno.cz/
64,CDZ Brno II (Práh jižní Morava),CDZ - Centrum duševního zdraví,"Jugoslávská 17, Zábrdovice (Brno-sever) (část)...",cdz@pnbrno.cz,737 390 737,https://www.pnbrno.cz/centrum-dusevniho-zdravi/
321,Odborné sociální poradenství - poradna PřES PRÁH,Odborné sociální poradenství,"Tuřanská 12, Brno-Tuřany (část) 620 00, Jihomo...",jana.wudyova@prah-brno.cz,734 850 546,http://www.prah-brno.cz/novinky/2-uncategorise...
492,"Pedagogicko-psychologická poradna Brno, příspě...",Pedagogicko-psychologické poradny,"Hybešova 253/15, Staré Brno, 602 00 Brno",hybesova@pppbrno.cz,541 422 822,https://www.pppbrno.cz/cs/
546,"Černý Vojtěch PhDr., Ph.D.",Psychoterapeuti,"Františkánská 6, 602 00 Brno",info@cervoy.cz,602 737 026,http://www.psychologiebrno.cz/
1428,Nízkoprahový klub Kumbál,Nízkoprahové zařízení,"Vídeňská 14/78, Brno-střed, Štýřice, 639 00 Brno",gajdziok.jan@brno.cz,543 210 670,http://www.brno.cz


In [10]:
not_matched = pd.read_csv("data/not_matched.csv")
urls = set(not_matched["web"])


# Email scraper

In [12]:

# Regular expressions for matching emails and phone numbers
email_regex = re.compile(r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+")
phone_regex = re.compile(r"\+?(\d[\d\-\(\) ]{9,}\d)")


# List to collect data
data = []

# Function to scrape a single URL
def scrape_website_main_page(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises an HTTPError if the response status code is 4XX or 5XX
        soup = BeautifulSoup(response.text, 'html.parser')

        emails = set(email_regex.findall(soup.text))
        phones = set(phone_regex.findall(soup.text))

        return emails, phones
    except requests.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return set(), set()

def scrape_website_contacts(base_url, path='', visited_urls=set()):
    full_url = urljoin(base_url, path)
    if full_url in visited_urls:
        return set(), set()  # Return empty sets if URL has already been visited
    visited_urls.add(full_url)

    try:
        response = requests.get(full_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        emails = set(email_regex.findall(soup.text))
        phones = set(phone_regex.findall(soup.text))

        # Checking for links that may contain contact info
        contact_links = []
        for link in soup.find_all('a', href=True):
            if 'kontakt' in link.text.lower() or 'contact' in link.text.lower() or 'kontakty' in link.text.lower() \
                or 'o nás' in link.text.lower():
                contact_href = urljoin(full_url, link['href'])
                if contact_href not in visited_urls:
                    contact_links.append(contact_href)

        # Scrape each potential contact page
        for link in contact_links:
            sub_emails, sub_phones = scrape_website_main_page(base_url, link, visited_urls)
            emails.update(sub_emails)
            phones.update(sub_phones)

        return emails, phones
    except requests.RequestException as e:
        #print(f"Error fetching {full_url}: {e}")
        return set(), set()

# Loop through the URLs and scrape each one
visited_urls = set()
for url in tqdm(urls):
    #print(f"Scraping {url}")
    
    emails, phones = scrape_website_main_page(url)
    # if  ('' in emails) or  ('' in phones):        
    #     emails_2, phones_2 = scrape_website_contacts(url, visited_urls=visited_urls)
    #     emails.add(emails_2)
    #     phones.add(phones_2)
    # Convert sets to comma-separated strings for storage
    #emails_str = ', '.join(emails)
    phones_str = ', '.join(phones)
    
    # Collect data
    #data.append({'Website': url, 'Emails': emails_str, 'Phone Numbers': phones_str})
    data.append({'Website': url, 'Phone Numbers': phones_str})
    #print("Done")
    #print("")
# Create DataFrame from collected data
df_res = pd.DataFrame(data)
df_res.to_csv('df_scraped.csv')
print(df)

  0%|          | 0/321 [00:00<?, ?it/s]

  1%|          | 2/321 [00:00<00:28, 11.13it/s]

Error fetching http://www.fokustabor.cz/centrum-dusevniho-zdravi-_-komunitni-tym-tabor: 404 Client Error: Not Found for url: http://www.fokustabor.cz/centrum-dusevniho-zdravi-_-komunitni-tym-tabor


  2%|▏         | 8/321 [00:11<11:04,  2.12s/it]

Error fetching https://www.cinka-decin.cz: HTTPSConnectionPool(host='www.cinka-decin.cz', port=443): Max retries exceeded with url: / (Caused by SSLError(CertificateError("hostname 'www.cinka-decin.cz' doesn't match either of '*.blueboard.cz', 'blueboard.cz'")))


...

Error fetching https://www.fokuslabe.cz/centra-dusevniho-zdravi/centrum-dusevniho-zdravi-chomutov/: 404 Client Error: Not Found for url: https://www.fokuslabe.cz/centra-dusevniho-zdravi/centrum-dusevniho-zdravi-chomutov/


 99%|█████████▉| 319/321 [18:02<00:06,  3.13s/it]

Error fetching hrdlickova@fokus-praha.cz: Invalid URL 'hrdlickova@fokus-praha.cz': No scheme supplied. Perhaps you meant https://hrdlickova@fokus-praha.cz?


100%|██████████| 321/321 [18:05<00:00,  3.38s/it]

                                     nazev                       Kategorie  \
0                 CDZ Hradec Králové (PDZ)  CDZ - Centrum duševního zdraví   
1                  CDZ Opava (Fokus Opava)  CDZ - Centrum duševního zdraví   
2                        CDZ Chrudim (PDZ)  CDZ - Centrum duševního zdraví   
3              CDZ Přerov (Duševní zdraví)  CDZ - Centrum duševního zdraví   
4     Psychiatrická ambulance Karlovy Vary         Psychiatrická ambulance   
...                                    ...                             ...   
1451                           NZDM Decima           Nízkoprahové zařízení   
1452                         NZDM Ymkárium           Nízkoprahové zařízení   
1453                            NZDM Dixie           Nízkoprahové zařízení   
1454               Nízkoprahový klub Orion           Nízkoprahové zařízení   
1455              Nízkoprahový klub U-kryt           Nízkoprahové zařízení   

                                                 Adresa  \
0   

In [13]:
df_res.to_csv('data/df_scraped_not_matched.csv')

In [63]:
df_res = pd.read_csv('df_scraped.csv')
df_res = df_res.rename(columns={'web': 'web', 'Telefon': 'Telefon'}).drop("Unnamed: 0", axis= 1)
df_res = df_res.rename(columns= {"Website":"web"})
df_res = df_res.rename(columns= {"Phone Numbers":"phones_scraped"})

df_res.head()

,web,phones_scraped
0,http://www.pdz.cz,NaN
1,http://www.medsix.cz/?option=com_content&view=...,NaN
2,https://www.osbonanza.cz,NaN
3,https://www.svetlokadanzs.cz,NaN
4,https://jindrichuvhradec.charita.cz/sluzba/852/,NaN


In [62]:
df_res

,web,Telefon
0,asistovanekontakty.cz,[603 739 658]
1,cdzostrava.cz,[595 223 166]
2,charita-most.cz,"[476 119 999, 476 119 999]"
3,csps-hk.cz,[607 248 533]
4,http:// www.psychamb.cz,[736 105 209]
...,...,...
866,psychoterapiehradeckralove.cz,[736 190 725]
867,salesianiteplice.cz,[739 959 566]
868,terapie-poradenstvi.wz.cz,[606 484 975]
869,tvuj-terapeut.cz,[775 505 315]


In [24]:
preview = df_res.merge(df_agg , on="web")
preview['phones_scraped'] = preview['phones_scraped'].str.split(', ')
preview

,web,phones_scraped,Telefon
0,http://www.pdz.cz,"[121 407 9329, 64 24 22 18, 420 466 400 116]","[778 526 216, 469 626 383, 774 915 755]"
1,http://www.medsix.cz/?option=com_content&view=...,"[483 341 232, 483 300 087, 483 310 919, 483 34...",[483 341 267]
2,https://www.osbonanza.cz,NaN,"[608 218 847, 608 218 847]"
3,https://www.svetlokadanzs.cz,NaN,"[725 556 109, 727 925 741, 702 042 647, 702 04..."
4,https://jindrichuvhradec.charita.cz/sluzba/852/,"[731 402 982, 731 604 554, 605 849 877]","[731 402 982, 605 849 877]"
...,...,...,...
864,https://pisek.charita.cz/jak-pomahame/oblastni...,NaN,[382 214 828]
865,http://ssp-ol.cz/,[585 427 141],[774 406 453]
866,http://www.psychoterapeuti.cz/adresar-psychote...,"[284 016 451, 736 660 315, 530 334 600, 775 18...",[466 611 601]
867,http://www.psychoterapeuti.cz/adresar-psychote...,"[284 016 451, 736 660 315, 530 334 600, 775 18...",[261 006 253]


In [28]:
type(preview['Telefon'][0])

list

# Exploding

In [61]:
df_res.head()

,web,phones_scraped
0,http://www.pdz.cz,NaN
1,http://www.medsix.cz/?option=com_content&view=...,NaN
2,https://www.osbonanza.cz,NaN
3,https://www.svetlokadanzs.cz,NaN
4,https://jindrichuvhradec.charita.cz/sluzba/852/,NaN


In [60]:
df_res['phones_scraped'] = df_res['phones_scraped'].str.split(', ')
df_agg = df_agg.rename(columns= {"Telefon":"phones_scraped"})
df_res_exp = df_res.explode('phones_scraped').reset_index(drop=True)
df_agg_exp = df_agg.explode('Telefon').reset_index(drop=True)
df_union = pd.concat([df_agg_exp, df_res_exp])

AttributeError: Can only use .str accessor with string values!

In [44]:
df_union.head()

,web,Telefon,phones_scraped
0,asistovanekontakty.cz,603 739 658,NaN
1,cdzostrava.cz,595 223 166,NaN
2,charita-most.cz,476 119 999,NaN
3,charita-most.cz,476 119 999,NaN
4,csps-hk.cz,607 248 533,NaN


In [45]:
df_union.groupby('web').count()

,Telefon,phones_scraped
web,,
asistovanekontakty.cz,1,0
cdzostrava.cz,1,0
charita-most.cz,2,0
csps-hk.cz,1,0
email,870,0
...,...,...
psychosomatika.info,1,0
psychoterapiehradeckralove.cz,1,0
salesianiteplice.cz,1,0


In [30]:
df_pivoted = preview.explode(['phones_scraped','Telefon']).reset_index(drop=True)
df_pivoted.head(30)

ValueError: columns must have matching element counts

In [15]:
preview.head(30)

,web,Phone Numbers,Telefon
0,https://www.zachranny-kruh.cz/,1 - 197-197,[353 230 000]
1,http://psych.fnplzen.cz/cs/node/839,"377 103 959, 377 441 850, 377 103 111, 377 401...",[377 103 111]
2,http://www.porceta.cz,,[777 882 002]
3,https://www.prevent99.cz/spot/,,[606 089 789]
4,http://www.detemzlin.cz/,577 922 056,[577 645 231]
5,https://www.homolka.cz/,,[257 271 111]
6,http://www.educozatec.cz/,,[603 253 634]
7,https://www.charitasluknov.cz,"602 236 279, 723 352 633","[734 799 081, 733 160 911]"
8,http://www.psychoterapeuti.cz/adresar-psychote...,"774 979 714, 257 322 366, 770 623 916, 607 669...","[233 351 741, 776 736 832]"
9,http://www.psychoterapeuti.cz/adresar-psychote...,"774 979 714, 257 322 366, 770 623 916, 607 669...",[777 808 424]
